In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [17]:
from crewai import Agent,Task,Crew,LLM
from dotenv import  load_dotenv
import os
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
serper_api_key=os.getenv("SERPER_API_KEY")

os.environ["OPENAI_API_KEY"] = "dummy"

llm = LLM(model="gemini/gemini-2.5-flash",
          provider="gemini",
          temperature=0.7,
          api_key=api_key)

In [18]:
# Define file paths for YAML configurations
import yaml
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [19]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [20]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
    llm=llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
    llm=llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
    llm=llm,
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True,
    manager_llm=llm
)

In [21]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



In [22]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 10a1c076-5b1b-46f7-b73f-742e1eab84ff                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Task: Carefully analyze the project_requirements for the Website project and break them down into individual   │
│  tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are      │
│  accounted for:                                                                                                 │
│                                                                                                                 │
│  - Create a responsive design that works well on desktop and mobile devices                                     │
│  - Implement a modern, visually appealing user interface with a clean look                                      │
│  - Develop a user-friendly navigation system with intuitive menu structure                                      │
│  - Include an "About Us" page highlighting the company's history and values                                     │
│  - Design a "Services" page showcasing the business's offerings with descriptions                               │
│  - Create a "Contact Us" page with a form and integrated map for communication                                  │
│  - Implement a blog section for sharing industry news and company updates                                       │
│  - Ensure fast loading times and optimize for search engines (SEO)                                              │
│  - Integrate social media links and sharing capabilities                                                        │
│  - Include a testimonials section to showcase customer feedback and build trust                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  As The Ultimate Project Planner, I have meticulously broken down the Website project for a small business      │
│  into actionable tasks, ensuring no detail is overlooked. The following plan aligns with the goal of creating   │
│  a responsive, modern, and user-friendly website, delivered on time and within scope.                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Project: Small Business Website Development**                                                                │
│  **Project Manager:** John Doe                                                                                  │
│  **Project Start Date:** Week 1, Day 1 (Assumed Monday)                                                         │
│  **Total Estimated Duration:** 8 Weeks                                                                          │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Comprehensive Task List**                                                                                │
│                                                                                                                 │
│  | Task ID | Task Name | Scope | Assigned To | Estimated Duration | Deliverables | Dependencies |               │
│  | :------ | :-------- | :---- | :---------- | :----------------- | :----------- | :----------- |               │
│  | **P1** | **Phase 1: Project Management & Discovery** | | | | | |                                             │
│  | P1.1 | Project Kick-off Meeting & Detailed Requirements | Formal meeting with client to finalize all         │
│  project requirements, scope, and expectations. Clarify content needs and brand identity. | John Doe | 1 Day |  │
│  Signed Project Scope Document, Detailed Content Requirements List | None |                                     │
│  | P1.2 | Technology Stack Selection & Environment Setup | Research and select appropriate technologies (e.g.,  │
│  CMS, hosting, frontend framework). Set up development, staging, and production environments. | John Doe, Jane  │
│  Doe | 2 Days | Tech Stack Document, Configured Dev Environment | P1.1 |                                        │
│  | P1.3 | Competitor & Audience Research | Analyze competitor websites and identify target audience             │
│  demographics and preferences to inform design and content strategy. | John Doe | 2 Days | Competitor Analysis  │
│  Report, Audience Persona Profiles | P1.1 |                                                                     │
│  | P1.4 | Content Strategy & SEO Keyword Research | Develop a content plan for all pages (About Us, Services,   │
│  Blog, Testimonials) and conduct keyword research to inform SEO. | John Doe, Jane Doe | 3 Days | Content Plan,  │
│  Primary Keyword List | P1.1, P1.3 |                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8d3e366f-efc6-47b7-b470-cf7fa64394fe                                                                     │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Estimation Analyst                                                                               │
│                                                                                                                 │
│  Task: Thoroughly evaluate each task in the Website project to estimate the time, resources, and effort         │
│  required. Use historical data, task complexity, and available resources to provide a realistic estimation for  │
│  each task.                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 877f4098-10a1-4850-a23e-ebbae29b37cd                                                                     │
│  Agent: Expert Estimation Analyst                                                                               │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Task: Strategically allocate tasks for the Website project to team members based on their skills,              │
│  availability, and current workload. Ensure that each task is assigned to the most suitable team member and     │
│  that the workload is evenly distributed.                                                                       │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### **Summary of Resource Allocation Rationale**                                                               │
│                                                                                                                 │
│  This resource allocation strategy aims to optimize team utilization, ensure tasks are assigned to the most     │
│  suitable team members, and balance workloads to maximize efficiency and project success, especially            │
│  addressing the heavy workload of the sole Software Engineer.                                                   │
│                                                                                                                 │
│  *   **John Doe (Project Manager):** As the Project Manager, John is primarily responsible for all project      │
│  management activities, client communication, strategic discovery (P1.1, P1.3, P1.5), and coordination (P2.4,   │
│  P7.7, P8.3, P8.4). To alleviate the significant burden on Jane Doe, the Software Engineer, John has also been  │
│  allocated tasks that require organizational skills, content understanding, and configuration rather than deep  │
│  technical coding. These include most of the content integration tasks (P5.1, P5.2, P5.3, P5.5) and basic SEO   │
│  configuration (P6.4, P6.5). This leverages his comprehensive project overview and client interaction           │
│  experience, ensuring a more balanced distribution of effort.                                                   │
│                                                                                                                 │
│  *   **Jane Doe (Software Engineer):** Jane is the core technical resource for the project. Her                 │
│  responsibilities encompass all aspects of development (P3.x, P4.x), environment setup (P1.2, P8.1), technical  │
│  SEO implementation (P1.4, P6.1, P6.2, P6.3), deployment (P8.2), and bug fixing (P7.6). She retains the blog    │
│  content population (P5.4) due to its potential for requiring intricate CMS interaction. By strategically       │
│  reallocating non-core development tasks to John, Jane's workload has been optimized to approximately 320       │
│  hours (equivalent to 8 weeks of full-time work), allowing her to focus her specialized skills on critical      │
│  technical delivery without being overburdened.                                                                 │
│                                                                                                                 │
│  *   **Bob Smith (Designer):** Bob is the specialist in UI/UX design. All tasks related to wireframing,         │
│  information architecture, design system creation, style guides, and high-fidelity mockups (P2.1, P2.2, P2.3)   │
│  are exclusively assigned to him. He collaborates with John Doe for client design review (P2.4) to present and  │
│  articulate design decisions, ensuring design quality and client satisfaction. His workload is concentrated in  │
│  the early design phases, aligning with the project's sequential flow.                                          │
│                                                                                                                 │
│  *   **Alice Johnson & Tom Brown (QA Engineers):** As the dedicated Quality Assurance team, Alice and Tom are   │
│  jointly responsible for all testing phases (P7.1-P7.6)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 209c8a4c-4645-4bf0-8918-753f5053a4d3                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 10a1c076-5b1b-46f7-b73f-742e1eab84ff                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### **Summary of Resource Allocation Rationale**                                                 │
│                                                                                                                 │
│  This resource allocation strategy aims to optimize team utilization, ensure tasks are assigned to the most     │
│  suitable team members, and balance workloads to maximize efficiency and project success, especially            │
│  addressing the heavy workload of the sole Software Engineer.                                                   │
│                                                                                                                 │
│  *   **John Doe (Project Manager):** As the Project Manager, John is primarily responsible for all project      │
│  management activities, client communication, strategic discovery (P1.1, P1.3, P1.5), and coordination (P2.4,   │
│  P7.7, P8.3, P8.4). To alleviate the significant burden on Jane Doe, the Software Engineer, John has also been  │
│  allocated tasks that require organizational skills, content understanding, and configuration rather than deep  │
│  technical coding. These include most of the content integration tasks (P5.1, P5.2, P5.3, P5.5) and basic SEO   │
│  configuration (P6.4, P6.5). This leverages his comprehensive project overview and client interaction           │
│  experience, ensuring a more balanced distribution of effort.                                                   │
│                                                                                                                 │
│  *   **Jane Doe (Software Engineer):** Jane is the core technical resource for the project. Her                 │
│  responsibilities encompass all aspects of development (P3.x, P4.x), environment setup (P1.2, P8.1), technical  │
│  SEO implementation (P1.4, P6.1, P6.2, P6.3), deployment (P8.2), and bug fixing (P7.6). She retains the blog    │
│  content population (P5.4) due to its potential for requiring intricate CMS interaction. By strategically       │
│  reallocating non-core development tasks to John, Jane's workload has been optimized to approximately 320       │
│  hours (equivalent to 8 weeks of full-time work), allowing her to focus her specialized skills on critical      │
│  technical delivery without being overburdened.                                                                 │
│                                                                                                                 │
│  *   **Bob Smith (Designer):** Bob is the specialist in UI/UX design. All tasks related to wireframing,         │
│  information architecture, design system creation, style guides, and high-fidelity mockups (P2.1, P2.2, P2.3)   │
│  are exclusively assigned to him. He collaborates with John Doe for client design review (P2.4) to present and  │
│  articulate design decisions, ensuring design quality and client satisfaction. His workload is concentrated in  │
│  the early design phases, aligning with the project's sequential flow.                                          │
│                                                                                                                 │
│  *   **Alice Johnson & Tom Brown (QA Engineers):** As 

In [24]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0065


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,43224,11368,0,31856,3


In [25]:
result.pydantic.dict()

{'tasks': [{'task_name': 'P1.1 Project Kick-off Meeting & Detailed Requirements',
   'estimated_time_hours': 8.0,
   'required_resources': ['John Doe']},
  {'task_name': 'P1.2 Technology Stack Selection & Environment Setup',
   'estimated_time_hours': 16.0,
   'required_resources': ['John Doe', 'Jane Doe']},
  {'task_name': 'P1.3 Competitor & Audience Research',
   'estimated_time_hours': 16.0,
   'required_resources': ['John Doe']},
  {'task_name': 'P1.4 Content Strategy & SEO Keyword Research',
   'estimated_time_hours': 24.0,
   'required_resources': ['John Doe', 'Jane Doe']},
  {'task_name': 'P1.5 Content Gathering & Asset Collection',
   'estimated_time_hours': 40.0,
   'required_resources': ['John Doe']},
  {'task_name': 'P2.1 Wireframing & Information Architecture',
   'estimated_time_hours': 32.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'P2.2 Design System & Style Guide Creation',
   'estimated_time_hours': 24.0,
   'required_resources': ['Bob Smith']},
  {'tas

In [26]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,P1.1 Project Kick-off Meeting & Detailed Requirements,8.000000,['John Doe']
1,P1.2 Technology Stack Selection & Environment Setup,16.000000,"['John Doe', 'Jane Doe']"
2,P1.3 Competitor & Audience Research,16.000000,['John Doe']
3,P1.4 Content Strategy & SEO Keyword Research,24.000000,"['John Doe', 'Jane Doe']"
4,P1.5 Content Gathering & Asset Collection,40.000000,['John Doe']
5,P2.1 Wireframing & Information Architecture,32.000000,['Bob Smith']
6,P2.2 Design System & Style Guide Creation,24.000000,['Bob Smith']
7,P2.3 High-Fidelity Mockups (Desktop & Mobile),40.000000,['Bob Smith']
8,P2.4 Client Design Review & Approval,16.000000,"['John Doe', 'Bob Smith']"
9,P3.1 Setup Frontend Framework & Responsive Base,24.000000,['Jane Doe']


In [27]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Phase 1: Project Management & Discovery,"['P1.1 Project Kick-off Meeting & Detailed Requirements', 'P1.2 Technology Stack Selection & Environment Setup', 'P1.3 Competitor & Audience Research', 'P1.4 Content Strategy & SEO Keyword Research', 'P1.5 Content Gathering & Asset Collection']"
1,Phase 2: UI/UX Design,"['P2.1 Wireframing & Information Architecture', 'P2.2 Design System & Style Guide Creation', 'P2.3 High-Fidelity Mockups (Desktop & Mobile)', 'P2.4 Client Design Review & Approval']"
2,Phase 3: Development - Frontend,"['P3.1 Setup Frontend Framework & Responsive Base', 'P3.2 Develop Global Navigation & Footer', 'P3.3 Develop ""About Us"" Page', 'P3.4 Develop ""Services"" Page', 'P3.5 Develop ""Contact Us"" Page (Frontend)', 'P3.6 Develop Blog Section (Frontend)', 'P3.7 Develop Testimonials Section', 'P3.8 Integrate Social Media Links & Sharing']"
3,Phase 4: Development - Backend & CMS (if applicable),"['P4.1 Implement Contact Form Submission Logic', 'P4.2 Integrate/Develop Blog CMS']"
4,Phase 5: Content Integration,"['P5.1 Populate ""About Us"" Content', 'P5.2 Populate ""Services"" Content', 'P5.3 Populate ""Contact Us"" Information', 'P5.4 Populate Initial Blog Posts', 'P5.5 Integrate Testimonials']"
5,Phase 6: Optimization & SEO,"['P6.1 On-Page SEO Implementation', 'P6.2 Image & Asset Optimization', 'P6.3 Code Minification & Performance Tuning', 'P6.4 Google Analytics & Search Console Setup', 'P6.5 Sitemap & Robots.txt Configuration']"
6,Phase 7: Quality Assurance & Testing,"['P7.1 Test Plan Creation', 'P7.2 Functional Testing', 'P7.3 Responsive & Cross-Browser Testing', 'P7.4 Performance & Usability Testing', 'P7.5 SEO Audit & Accessibility Check', 'P7.6 Bug Fixing & Regression Testing', 'P7.7 User Acceptance Testing (UAT)']"
7,Phase 8: Deployment & Post-Launch,"['P8.1 Production Server Setup & Configuration', 'P8.2 Final Code Deployment', 'P8.3 Post-Launch Monitoring & Verification', 'P8.4 Client Handoff & Training']"
